In [31]:

import numpy as np
#import matplotlib.pyplot as plt
import datetime

# Importations Qiskit mises à jour
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.primitives import Sampler as BaseSampler
from qiskit.quantum_info import Statevector
from qiskit.result import QuasiDistribution
from qiskit.utils import algorithm_globals

ModuleNotFoundError: No module named 'qiskit.algorithms'

In [ ]:
# set number of assets (= number of qubits)
num_assets = 4
seed = 123

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2016, 1, 1),
    end=datetime.datetime(2016, 1, 30),
    seed=seed,
)
data.run()
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix(

In [ ]:
# plot sigma
plt.imshow(sigma, interpolation="nearest")
plt.show()

In [ ]:
q = 0.5  # set risk factor
budget = num_assets // 2  # set budget
penalty = num_assets  # set parameter to scale the budget penalty term

portfolio = PortfolioOptimization(
    expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget
)
qp = portfolio.to_quadratic_program()
qp

In [ ]:
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [ ]:
exact_mes = NumPyMinimumEigensolver()
exact_eigensolver = MinimumEigenOptimizer(exact_mes)

result = exact_eigensolver.solve(qp)

print_result(result)

In [ ]:
from qiskit_algorithms.utils import algorithm_globals

algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(num_assets, "ry", "cz", reps=3, entanglement="full")
svqe_mes = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=cobyla)
svqe = MinimumEigenOptimizer(svqe_mes)
result = svqe.solve(qp)

print_result(result)

In [ ]:
algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=3)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

print_result(result

In [ ]:
import tutorial_magics

%qiskit_version_table
%qiskit_copyright

In [30]:
import numpy as np
#import matplotlib.pyplot as plt
import datetime

# Updated Qiskit imports
from qiskit.circuit.library import TwoLocal
#f rom qiskit.primitives import  BaseSamplerV1 as Sampler 
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.quantum_info import Statevector
from qiskit.result import QuasiDistribution
from qiskit.utils import algorithm_globals

# Configuration
num_assets = 4
seed = 123
algorithm_globals.random_seed = seed

# Generate random portfolio data
stocks = [f"TICKER{i}" for i in range(num_assets)]
start_date = datetime.datetime(2016, 1, 1)
end_date = datetime.datetime(2016, 1, 30)

# Function to generate random return data
def generate_random_data(num_assets, seed):
    np.random.seed(seed)
    # Generate random returns
    returns = np.random.normal(0.001, 0.02, (30, num_assets))
    # Calculate mean returns vector
    mu = np.mean(returns, axis=0)
    # Calculate covariance matrix
    sigma = np.cov(returns, rowvar=False)
    return mu, sigma

# Generate data
mu, sigma = generate_random_data(num_assets, seed)

# Display covariance matrix
plt.figure(figsize=(8, 6))
plt.imshow(sigma, interpolation="nearest")
plt.colorbar()
plt.title("Covariance Matrix")
plt.tight_layout()
plt.show()

# Parameters for the optimization problem
lambda_0 = 1.0    # Weight for risk minimization
lambda_1 = 2.0    # Penalty parameter for budget constraint
lambda_2 = 2.0    # Penalty parameter for return constraint
n = num_assets // 2   # Target number of assets to select
R_star = 0.002    # Minimum required return
K = 3             # Number of auxiliary variables for the return constraint

from qiskit_optimization import QuadraticProgram

# Create a custom QUBO problem
def create_portfolio_qubo(mu, sigma, n, R_star, lambda_0, lambda_1, lambda_2, K):
    """
    Create a QUBO formulation for portfolio optimization:
    min λ0xT Σx + λ1 (∑xi − n)² + λ2 (μT x − R∗ − ∑2^k*yk)²
    
    Args:
        mu: Expected returns vector
        sigma: Covariance matrix
        n: Target portfolio size
        R_star: Minimum required return
        lambda_0, lambda_1, lambda_2: Penalty parameters
        K: Number of auxiliary variables
    """
    total_vars = num_assets + K  # Including auxiliary variables
    qp = QuadraticProgram(name="Portfolio Optimization QUBO")
    
    # Add binary variables for asset selection
    for i in range(num_assets):
        qp.binary_var(name=f'x{i}')
    
    # Add auxiliary binary variables for return constraint
    for k in range(K):
        qp.binary_var(name=f'y{k}')
    
    # Create the QUBO objective function
    # Start with risk term: λ0 * xT Σ x
    for i in range(num_assets):
        for j in range(num_assets):
            qp.quadratic_constraint(linear={}, quadratic={(f'x{i}', f'x{j}'): lambda_0 * sigma[i][j]}, sense='==', rhs=0, name=f'risk_{i}_{j}')
    
    # Add budget constraint penalty: λ1 * (∑xi - n)²
    # Expanding: λ1 * [(∑xi)² - 2n(∑xi) + n²]
    # The n² term is constant and can be ignored in optimization
    
    # (∑xi)² term
    for i in range(num_assets):
        for j in range(num_assets):
            qp.quadratic_constraint(linear={}, quadratic={(f'x{i}', f'x{j}'): lambda_1}, sense='==', rhs=0, name=f'budget_quad_{i}_{j}')
    
    # -2n(∑xi) term
    for i in range(num_assets):
        qp.linear_constraint(linear={f'x{i}': -2 * lambda_1 * n}, sense='==', rhs=0, name=f'budget_lin_{i}')
    
    # Add return constraint penalty: λ2 * (μT x - R* - ∑2^k*yk)²
    # Expanding: λ2 * [(μT x)² - 2(μT x)(R* + ∑2^k*yk) + (R* + ∑2^k*yk)²]
    
    # (μT x)² term
    for i in range(num_assets):
        for j in range(num_assets):
            qp.quadratic_constraint(linear={}, quadratic={(f'x{i}', f'x{j}'): lambda_2 * mu[i] * mu[j]}, sense='==', rhs=0, name=f'return_quad_xx_{i}_{j}')
    
    # -2(μT x)(R*) term
    for i in range(num_assets):
        qp.linear_constraint(linear={f'x{i}': -2 * lambda_2 * mu[i] * R_star}, sense='==', rhs=0, name=f'return_lin_x_R_{i}')
    
    # -2(μT x)(∑2^k*yk) term
    for i in range(num_assets):
        for k in range(K):
            qp.quadratic_constraint(linear={}, quadratic={(f'x{i}', f'y{k}'): -2 * lambda_2 * mu[i] * (2**k)}, sense='==', rhs=0, name=f'return_quad_xy_{i}_{k}')
    
    # (∑2^k*yk)² term
    for k1 in range(K):
        for k2 in range(K):
            qp.quadratic_constraint(linear={}, quadratic={(f'y{k1}', f'y{k2}'): lambda_2 * (2**k1) * (2**k2)}, sense='==', rhs=0, name=f'return_quad_yy_{k1}_{k2}')
    
    # 2*R*(∑2^k*yk) term
    for k in range(K):
        qp.linear_constraint(linear={f'y{k}': 2 * lambda_2 * R_star * (2**k)}, sense='==', rhs=0, name=f'return_lin_y_R_{k}')
    
    # R*² term is constant and can be ignored for optimization
    
    # Set minimization objective
    qp.minimize()
    
    return qp

# Create the QUBO problem
qp = create_portfolio_qubo(mu, sigma, n, R_star, lambda_0, lambda_1, lambda_2, K)

print(qp)

# Function to display results
def print_result(result, mu, sigma, n, R_star):
    """Print and analyze optimization results"""
    selection = result.x
    value = result.fval
    
    # Extract asset selection variables (first num_assets variables)
    x = selection[:num_assets]
    y = selection[num_assets:]
    
    print(f"Optimal portfolio selection: {x}")
    print(f"Auxiliary variables y: {y}")
    print(f"Objective value: {value:.6f}")
    
    # Calculate actual portfolio metrics
    portfolio_size = np.sum(x)
    portfolio_return = np.dot(mu, x)
    portfolio_risk = np.dot(x, np.dot(sigma, x))
    
    print("\nPortfolio Analysis:")
    print(f"Portfolio size: {portfolio_size} (target: {n})")
    print(f"Expected return: {portfolio_return:.6f} (minimum required: {R_star:.6f})")
    print(f"Portfolio risk: {portfolio_risk:.6f}")
    
    # Check constraints
    budget_penalty = lambda_1 * (portfolio_size - n)**2
    auxiliary_sum = sum(2**k * y[k] for k in range(len(y)))
    return_penalty = lambda_2 * (portfolio_return - R_star - auxiliary_sum)**2
    
    print("\nConstraint Analysis:")
    print(f"Budget constraint penalty: {budget_penalty:.6f}")
    print(f"Return constraint penalty: {return_penalty:.6f}")
    
    # Extract probabilities from result if available
    if hasattr(result, 'min_eigen_solver_result'):
        eigenstate = result.min_eigen_solver_result.eigenstate
        if isinstance(eigenstate, QuasiDistribution):
            probabilities = eigenstate.binary_probabilities()
        elif isinstance(eigenstate, Statevector):
            probabilities = {format(i, f'0{num_assets + K}b')[::-1]: np.abs(v)**2 
                            for i, v in enumerate(eigenstate.data) if np.abs(v)**2 > 1e-6}
        else:
            probabilities = {k: np.abs(v)**2 for k, v in eigenstate.to_dict().items()}
        
        # Display top solutions
        print("\n--- Top Solutions (Probability Distribution) ---")
        sorted_probs = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)
        for i, (bitstr, prob) in enumerate(sorted_probs[:5]):
            if isinstance(bitstr, str):
                bits = np.array([int(b) for b in bitstr])
            else:
                bits = np.array([int(b) for b in list(reversed(bin(bitstr)[2:].zfill(num_assets + K)))])
            
            x_bits = bits[:num_assets]
            y_bits = bits[num_assets:] if len(bits) > num_assets else []
            
            port_size = np.sum(x_bits)
            port_return = np.dot(mu, x_bits[:num_assets])
            port_risk = np.dot(x_bits[:num_assets], np.dot(sigma, x_bits[:num_assets]))
            
            print(f"Solution {i+1}: x={x_bits}, y={y_bits}")
            print(f"  Probability: {prob:.4f}")
            print(f"  Size: {port_size}, Return: {port_return:.6f}, Risk: {port_risk:.6f}")

# 1. Exact solution with NumPyMinimumEigensolver
print("\n=== Solution with NumPyMinimumEigensolver ===")
exact_mes = NumPyMinimumEigensolver()
exact_eigensolver = MinimumEigenOptimizer(exact_mes)
exact_result = exact_eigensolver.solve(qp)
print_result(exact_result, mu, sigma, n, R_star)

# 2. Solution with SamplingVQE
print("\n=== Solution with SamplingVQE ===")
algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=500)

# Use Qiskit's base sampler
qiskit_sampler = BaseSampler()
# Create variational circuit
ry = TwoLocal(num_assets + K, "ry", "cz", reps=3, entanglement="full")

# Configure SamplingVQE with sampler
svqe_mes = SamplingVQE(sampler=qiskit_sampler, ansatz=ry, optimizer=cobyla)
svqe = MinimumEigenOptimizer(svqe_mes)

# Solve the problem
svqe_result = svqe.solve(qp)
print_result(svqe_result, mu, sigma, n, R_star)

# 3. Solution with QAOA
print("\n=== Solution with QAOA ===")
algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)

# Configure QAOA with sampler
qaoa_mes = QAOA(sampler=qiskit_sampler, optimizer=cobyla, reps=3)
qaoa = MinimumEigenOptimizer(qaoa_mes)

# Solve the problem
qaoa_result = qaoa.solve(qp)
print_result(qaoa_result, mu, sigma, n, R_star)

ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qiskit\primitives\__init__.py)